In [1]:

# Load in AAM
from askametric.query_processor.query_processor import LLMQueryProcessor

from dotenv import load_dotenv

from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.asyncio import (
    AsyncSession,
    create_async_engine,
)

# Load Environment Variables
load_dotenv(".env")

True

## Investigating issues with the JSON columns in SQLite (reproduces same error as with Postgres db)

### UPDATE: We have updated the pipeline to tell the LLM exactly which database type it is connected to. This seems to solve the issue

In [2]:
# Parameters
metric_db_id = "tn_covid_cases_11_may"
llm = "gpt-4o"
guardrails_llm = "gpt-4o"
sys_message = "Government and health officials in Tamil Nadu, India will ask you questions. You need to help them manage COVID cases and the availablity of beds in health facilities."
original_db_description = "- bed_vacancies_clinics_may_11: Each row identifies a district and the beds earmarked, occupied and available for COVID cases in the district clinics.\
- bed_vacancies_health_centers_and_district_hospitals_11_may: Each row identifies a district and the beds earmarked, occupied and available, with and without oxygen supply, and with and without ICU support, for COVID cases in the disctrict health centers and hospitals.\
- covid_cases_11_may: Each row identifies a district and the number of people who received treatment, were discharged and died due to COVID.\
"

updated_sys_message = "Government and health officials in Tamil Nadu, India will ask you questions. You need to help them manage COVID cases and the availablity of beds in health facilities.\
    The database contains JSON columns. Use `json_extract` to extract the required information."
updated_db_description = "- bed_vacancies_clinics_11_may: Each row identifies a district and a JSON object including the beds earmarked, occupied and available for COVID cases in the district clinics.\
- bed_vacancies_health_centers_and_district_hospitals_11_may: Each row identifies a district and the beds earmarked, occupied and available, with and without oxygen supply, and with and without ICU support, for COVID cases in the disctrict health centers and hospitals.\
- covid_cases_11_may: Each row identifies a district and the number of people who received treatment, were discharged and died due to COVID.\
"

num_common_values = 10
indicator_vars=""

In [3]:
""" Get assession for db schema and aam """
aengine = create_async_engine(
            url="sqlite+aiosqlite:///./tn_covid_cases_11_may_with_jsonb.sqlite"
        )

async_session = sessionmaker(
    bind=aengine,
    class_=AsyncSession,
    expire_on_commit=False
)

In [4]:
question = "Which district has the most available beds in clinics?"
async with async_session() as session:
    qp = LLMQueryProcessor(
        {"query_text": question, "query_metadata": {}},
        session,
        metric_db_id,
        "sqlite",
        "gpt-4o",
        "gpt-4o",
        sys_message,
        updated_db_description,
        column_description="",
        num_common_values=num_common_values,
        indicator_vars=indicator_vars
    )
    await qp.process_query()

print(qp.final_answer)
print(qp.sql_query)

The district with the most available beds in clinics is Chennai, with 3,472 vacant beds. This information was determined by checking the number of vacant beds in each district and finding that Chennai has the highest number.
SELECT district_name, json_extract(available_beds, '$.vacant_beds') AS vacant_beds FROM bed_vacancies_clinics_11_may ORDER BY vacant_beds DESC LIMIT 1;


In [5]:
# retry with updated description and system message
question = "Which district had the most available beds in clinics" #<--need to remove question mark to get around the cache
async with async_session() as session:
    qp = LLMQueryProcessor(
        {"query_text": question, "query_metadata": {}},
        session,
        metric_db_id,
        "sqlite",
        "gpt-4o",
        "gpt-4o",
        updated_sys_message,
        updated_db_description,
        column_description="",
        num_common_values=num_common_values,
        indicator_vars=indicator_vars
    )
    await qp.process_query()

print(qp.final_answer)
print(qp.sql_query)

The district with the most available beds in clinics is Chennai, with 3472 vacant beds. This information was obtained by checking the number of vacant beds in each district and identifying the one with the highest count.
SELECT district_name, json_extract(available_beds, '$.vacant_beds') AS vacant_beds FROM bed_vacancies_clinics_11_may ORDER BY vacant_beds DESC LIMIT 1;
